# Entertainment Genres: App

# Setting Up Necessary Things

In [ ]:
# Jupyter Notebook Magic Command - Auto Reloading
%reload_ext autoreload
%autoreload 2

# Jupyter Notebook Magic Command - Inline Plotting
%matplotlib inline

In [ ]:
# Ignore All Warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/All Entertainment Classification

/content/drive/MyDrive/All Entertainment Classification


# Necessary Imports

In [ ]:
! pip install -q transformers[sentencepiece] fastai onnxruntime gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.8 MB/s eta 0:00:00


In [ ]:
import numpy as np

import torch

from transformers import AutoTokenizer

import onnxruntime as rt

import gradio as gr

import json

# Modeling

In [ ]:
model_path = "models/entertainment-genre.onnx"

In [ ]:
with open("genre_types_encoded.json", "r") as file:
  categories = json.load(file)

In [ ]:
categories

{'Fiction': 0,
 'pop': 1,
 'Thriller': 2,
 'Mystery': 3,
 'Crime': 4,
 'Drama': 5,
 'Indie': 6,
 'Action': 7,
 'Adventure': 8,
 'Romance': 9,
 'Casual': 10,
 'Comedy': 11,
 'Strategy': 12,
 'Simulation': 13,
 'Historical': 14,
 'Sci-Fi': 15,
 'Nonfiction': 16,
 'Horror': 17,
 'Fantasy': 18,
 'Young Adult': 19}

In [ ]:
inf_session = rt.InferenceSession(model_path)
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [ ]:
def get_top_label(cat_dict, idx):
  for key, value in cat_dict.items():
    if idx == value:
      return key

In [ ]:
def get_top_probs(cat_probs, idx):
  return cat_probs[idx]

In [ ]:
def entertainment_genres(description):
  input_ids = tokenizer(description)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  top_3_indices = sorted(range(len(probs[0])), key=lambda i: probs[0][i], reverse=True)[:3]
  cat_prob = torch.sigmoid(torch.FloatTensor(probs))[0]
  print(cat_prob)

  top_labels = []
  for i in top_3_indices:
    top_labels.append(get_top_label(categories, i))

  top_probs = []
  for i in top_3_indices:
    top_probs.append(get_top_probs(cat_prob, i))
  return dict(zip(top_labels, map(float, top_probs)))

In [ ]:
desc = "When the menace knoical tests of his ability to fight injustice."

In [ ]:
entertainment_genres(desc)

tensor([0.1486, 0.0482, 0.0496, 0.0695, 0.0491, 0.1482, 0.1592, 0.2170, 0.2019,
        0.1561, 0.0923, 0.1385, 0.0429, 0.0462, 0.0411, 0.0553, 0.0594, 0.0602,
        0.1754, 0.0550])


{'Action': 0.2170487493276596,
 'Adventure': 0.2019059658050537,
 'Fantasy': 0.1754470318555832}

# Gradio

In [ ]:
example = [
    ["March Of Soldiers is a real time strategy single player , It is a military game based on the player's skill and the strength of his financial economy"],
    ["When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice."]
]

In [ ]:
label = gr.outputs.Label(num_top_classes=3)

iface = gr.Interface(fn=entertainment_genres, inputs="text", outputs=label, examples=example)
iface.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://10c50415ec8d731e89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
